In [1]:
import time

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()



X, y = diabetes.data, diabetes.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

def print_accuracy(f):
    print(f"Root mean squared test error = {np.sqrt(np.mean((f(X_test) - y_test) ** 2))}")
    time.sleep(0.5)  # to let the print get out before any progress bars

print(X.shape)


(442, 10)


In [2]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier(max_depth=2, random_state=0)
rforest.fit(X_train, y_train)

print_accuracy(rforest.predict)

Root mean squared test error = 83.00338425481704


In [17]:
sample_original = X_test[1]
original_class = rforest.predict([sample_original])[0]
def Tabular_Mask(sample, n_samples=50, X=None, feature_names=None):
    if X is None:
        raise ValueError("X must be provided to determine feature ranges")
    
    masks = np.tile(sample, (n_samples, 1))
    change_masks = np.tile(sample, (n_samples, 1))
    
    # Determine the index of the "sex" feature
    sex_feature_index = None
    if feature_names is not None:
        for i, feature_name in enumerate(feature_names):
            if feature_name.lower() == 'sex':
                sex_feature_index = i
                break
    
    if sex_feature_index is None:
        print("Warning: Sex feature not found in the dataset. Treating all features as continuous.")
    
    for i in range(n_samples):
        feature_to_change = np.random.randint(0, len(sample))
        
        if feature_to_change == sex_feature_index:
            # For the sex feature, randomly choose from the available values
            sex_values = np.unique(X[:, sex_feature_index])
            new_value = np.random.choice(sex_values)
        else:
            # For other features, use the original uniform distribution
            min_val = X[:, feature_to_change].min()
            max_val = X[:, feature_to_change].max()
            new_value = np.random.uniform(min_val, max_val)
        
        masks[i, feature_to_change] = new_value
        change_masks[i, feature_to_change] = new_value
    
    return masks, change_masks

def create_test_suit(sample, n_samples=100, X=None, feature_names=None):
    correct_mutants = []
    incorrect_mutants = []
    correct_masks = []
    incorrect_masks = []
    clf = rforest
    original_class = clf.predict([sample])[0]
    
    while len(correct_mutants) < n_samples // 2 or len(incorrect_mutants) < n_samples // 2:
        masks, change_masks = Tabular_Mask(sample, n_samples=10, X=X, feature_names=feature_names)
        for mask, change_mask in zip(masks, change_masks):
            predicted_class = clf.predict([mask])[0]
            if predicted_class == original_class and len(correct_mutants) < n_samples // 2:
                correct_mutants.append(mask)
                correct_masks.append(change_mask)
            elif predicted_class != original_class and len(incorrect_mutants) < n_samples // 2:
                incorrect_mutants.append(mask)
                incorrect_masks.append(change_mask)
    
    Test_Suit = np.vstack((correct_mutants[:n_samples//2], incorrect_mutants[:n_samples//2]))
    Test_Suit_Masks = np.vstack((correct_masks[:n_samples//2], incorrect_masks[:n_samples//2]))
    
    # Shuffle both Test_Suit and Test_Suit_Masks in the same order
    shuffle_indices = np.random.permutation(len(Test_Suit))
    Test_Suit = Test_Suit[shuffle_indices]
    Test_Suit_Masks = Test_Suit_Masks[shuffle_indices]
    
    return Test_Suit, Test_Suit_Masks

# Assuming X_test is your test dataset and feature_names is a list of feature names
sample_original = X_test[1]
feature_names = ['feature1', 'feature2', 'sex', 'feature4', ...]  # Replace with your actual feature names

# Create the Test_Suit
Test_Suit, Test_Suit_Masks = create_test_suit(sample_original, X=X_test, feature_names=feature_names)
    

In [18]:
# Print the original sample and its predicted class
print("Original sample:")
print(sample_original)
clf = rforest
print(f"Predicted class for original sample: {original_class})")

print("\nTest_Suit mutants and their predictions:")
for i, mutant in enumerate(Test_Suit):
    print(f"Mutant {i}:")
    print(mutant)
    predicted_class = clf.predict([mutant])[0]

    print(f"Changed feature: {np.where(mutant != sample_original)[0][0]}")
    print(f"Correctly classified: {predicted_class == original_class}")
    print()

# Count the occurrences of each predicted class in Test_Suit
test_suit_predictions = clf.predict(Test_Suit)
unique, counts = np.unique(test_suit_predictions, return_counts=True)
print("\nPrediction distribution for Test_Suit:")


# Verify the balance of Test_Suit
correct_count = sum(clf.predict(Test_Suit) == original_class)
print(f"\nCorrectly classified mutants in Test_Suit: {correct_count}")
print(f"Incorrectly classified mutants in Test_Suit: {len(Test_Suit) - correct_count}")

# Print class name mapping for reference


# Print feature names for reference
print("\nFeature names:")
for i, name in enumerate(diabetes.feature_names):
    print(f"Feature {i}: {name}")

Original sample:
[-0.01277963 -0.04464164  0.06061839  0.05285804  0.04796534  0.02937467
 -0.01762938  0.03430886  0.07020738  0.00720652]
Predicted class for original sample: 220.0)

Test_Suit mutants and their predictions:
Mutant 0:
[-0.01277963 -0.04464164  0.06061839  0.05285804  0.04796534  0.02937467
 -0.01762938  0.03430886  0.0012956   0.00720652]
Changed feature: 8
Correctly classified: False

Mutant 1:
[-0.01277963 -0.04464164  0.06061839  0.05285804  0.04796534  0.02937467
 -0.01762938  0.03430886 -0.0076726   0.00720652]
Changed feature: 8
Correctly classified: False

Mutant 2:
[-0.01277963 -0.04464164  0.06061839  0.05285804  0.04796534  0.02937467
  0.08565892  0.03430886  0.07020738  0.00720652]
Changed feature: 6
Correctly classified: True

Mutant 3:
[-0.01277963 -0.04464164  0.06061839  0.05285804  0.04796534  0.06402906
 -0.01762938  0.03430886  0.07020738  0.00720652]
Changed feature: 5
Correctly classified: True

Mutant 4:
[-0.01277963 -0.04464164  0.06061839  0.05

In [19]:
def calculate_feature_relevance(Test_Suit, sample_original, clf):
    n_features = len(sample_original)
    Et = np.zeros(n_features)
    Ef = np.zeros(n_features)
    Nt = np.zeros(n_features)
    Nf = np.zeros(n_features)

    original_prediction = clf.predict([sample_original])[0]

    for mutant in Test_Suit:
        mutant_prediction = clf.predict([mutant])[0]
        for feature in range(n_features):
            if mutant[feature] == sample_original[feature]:
                if mutant_prediction == original_prediction:
                    Et[feature] += 1
                else:
                    Ef[feature] += 1
            else:
                if mutant_prediction == original_prediction:
                    Nt[feature] += 1
                else:
                    Nf[feature] += 1

    return Et, Ef, Nt, Nf

# Calculate feature relevance
Et, Ef, Nt, Nf = calculate_feature_relevance(Test_Suit, sample_original, clf)

In [20]:
# Calculate Wong1 feature importance
def calculate_ochiai(Ef, Ep, Nf, Np):
    numerator = Ef
    denominator = np.sqrt((Ef + Nf) * (Ef + Ep))
    return np.divide(numerator, denominator, where=denominator!=0)

def calculate_tarantula(Ef, Ep, Nf, Np):
    numerator = np.divide(Ef, Ef + Nf, where=(Ef + Nf)!=0)
    denominator = numerator + np.divide(Ep, Ep + Np, where=(Ep + Np)!=0)
    return np.divide(numerator, denominator, where=denominator!=0)

def calculate_zoltar(Ef, Ep, Nf, Np):
    epsilon = 1e-10  # Small value to prevent division by zero
    return Ef / (Ef + Nf + Ep + (10000 * Nf * Ep / (Ef + epsilon)))

def calculate_wong1(Et, Ef):
    return Et - Ef

formulas = {
    "Wong1": calculate_wong1(Et, Ef),
    "Ochiai": 1- calculate_ochiai(Ef, Et, Nf, Nt),
    "Tarantula": 1- calculate_tarantula(Ef, Et, Nf, Nt),
    "Zoltar": 1- calculate_zoltar(Ef, Et, Nf, Nt)
}


# Normalize scores for each formula
normalized_formulas = {}
for formula_name, scores in formulas.items():
    normalized_scores = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
    normalized_formulas[f"{formula_name}_normalized"] = normalized_scores

# Combine raw and normalized scores
all_scores = {**formulas, **normalized_formulas}

# Print feature importance for each formula
for formula_name, scores in all_scores.items():
    print(f"\n{formula_name} Feature Importance:")
    for i, feature_name in enumerate(diabetes.feature_names):
        print(f"{feature_name}: {scores[i]:.4f}")

    # Sort features by importance
    sorted_indices = np.argsort(scores)[::-1]
    print(f"\nFeatures sorted by {formula_name} importance (most to least important):")
    for i in sorted_indices:
        print(f"{diabetes.feature_names[i]}: {scores[i]:.4f}")

# Prepare the data for plotting
data = [
    {
        "name": diabetes.feature_names[i],
        **{f"{formula_name}": scores[i] for formula_name, scores in all_scores.items()}
    }
    for i in range(len(diabetes.feature_names))
]

print("\nData prepared for plotting:")
print(data)


Wong1 Feature Importance:
age: 5.0000
sex: -3.0000
bmi: -5.0000
bp: -5.0000
s1: -5.0000
s2: 8.0000
s3: -1.0000
s4: -2.0000
s5: 9.0000
s6: -1.0000

Features sorted by Wong1 importance (most to least important):
s5: 9.0000
s2: 8.0000
age: 5.0000
s6: -1.0000
s3: -1.0000
s4: -2.0000
sex: -3.0000
s1: -5.0000
bp: -5.0000
bmi: -5.0000

Ochiai Feature Importance:
age: 0.3625
sex: 0.2820
bmi: 0.3025
bp: 0.2745
s1: 0.3025
s2: 0.4222
s3: 0.3181
s4: 0.2857
s5: 0.4256
s6: 0.3254

Features sorted by Ochiai importance (most to least important):
s5: 0.4256
s2: 0.4222
age: 0.3625
s6: 0.3254
s3: 0.3181
s1: 0.3025
bmi: 0.3025
s4: 0.2857
sex: 0.2820
bp: 0.2745

Tarantula Feature Importance:
age: 0.5275
sex: 0.4845
bmi: 0.4713
bp: 0.4737
s1: 0.4713
s2: 0.5488
s3: 0.4945
s4: 0.4898
s5: 0.5542
s6: 0.4944

Features sorted by Tarantula importance (most to least important):
s5: 0.5542
s2: 0.5488
age: 0.5275
s3: 0.4945
s6: 0.4944
s4: 0.4898
sex: 0.4845
bp: 0.4737
s1: 0.4713
bmi: 0.4713

Zoltar Feature Importanc

In [21]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display

def plot_feature_importance(data, view='Raw', measure='Wong1'):
    fig, ax = plt.subplots(figsize=(14, 8))
    fig.suptitle('Feature Importance Comparison', fontsize=16)

    # Sort data by absolute raw importance of the selected measure
    sorted_data = sorted(data, key=lambda x: abs(x[f'{measure}{"" if view == "Raw" else "_normalized"}']), reverse=True)
    names = [item['name'] for item in sorted_data]
    index = np.arange(len(names))

    # Use coolwarm colormap
    cmap = plt.get_cmap('coolwarm')

    values = [item[f'{measure}{"" if view == "Raw" else "_normalized"}'] for item in sorted_data]
    
    # Create a normalize object for this measure
    if view == 'Raw':
        norm = Normalize(vmin=min(values), vmax=max(values))
    else:
        norm = Normalize(vmin=0, vmax=1)
    
    # Map each value to a color
    colors = [cmap(norm(value)) for value in values]
    
    # Plot the bars
    bars = ax.barh(index, values, 0.8, label=measure, color=colors, alpha=0.7)
    
    # Add a colorbar for this measure
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.02)
    cbar.set_label(f'{measure} Importance', rotation=270, labelpad=15)

    ax.set_yticks(index)
    ax.set_yticklabels(names)
    ax.set_title(f'{view} Importance - {measure}')
    ax.set_xlabel('Importance')

    if view == 'Raw':
        ax.axvline(x=0, color='gray', linestyle='--')
    else:
        ax.set_xlim(0, 1)

    plt.tight_layout()
    plt.show()

# Create interactive widgets
view_widget = widgets.RadioButtons(options=['Raw', 'Normalized'], description='View:')
measure_widget = widgets.RadioButtons(
    options=['Wong1', 'Ochiai', 'Tarantula', 'Zoltar'],
    description='Measure:'
)

# Combine widgets
widgets_combined = widgets.VBox([view_widget, measure_widget])

# Create interactive plot
interactive_plot = interactive(plot_feature_importance,
                               data=fixed(data),
                               view=view_widget,
                               measure=measure_widget)

# Display widgets and plot
display(widgets_combined, interactive_plot.children[-1])

Output()